# ジンバルロック

吉田勝俊（宇都宮大学）

## 参考情報
- [ipywidgetsでインタラクティブなグラフを作る - Qiita](https://qiita.com/studio_haneya/items/adbaa01b637e7e699e75)
- [Widget List &mdash; Jupyter Widgets 8.0.0a5 documentation](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)

In [ ]:
import numpy as np                        #数値計算ライブラリ
from math import sin, cos, pi             #低機能だが計算が速い数学関数
import matplotlib.pyplot as plt           #描画ライブラリ
from mpl_toolkits.mplot3d import Axes3D   #3次元座標用のモジュール
from mpl_toolkits.mplot3d.art3d import Poly3DCollection #3次元ポリゴン用のモジュール
import ipywidgets as ipw                  #対話的処理のモジュール
#↓Colab用の設定（グラフィックのインライン表示）
%matplotlib inline

## 3次元の回転行列

### 基本となる回転行列（回転軸$\boldsymbol{a}$，角度$\theta$）

In [ ]:
def Rotation(angle, axis):

  a1, a2, a3 = axis #回転軸の成分
  aa1 = a1*a1 #2乗を作っておく
  aa2 = a2*a2
  aa3 = a3*a3
  C = cos(angle)
  S = sin(angle)

  R = np.array([
    [ aa1 + (1-aa1)*C,    a1*a2*(1-C) - a3*S, a1*a3*(1-C) + a2*S ],              
    [ a1*a2*(1-C) + a3*S, aa2 + (1-aa2)*C,    a2*a3*(1-C) - a1*S ],              
    [ a1*a3*(1-C) - a2*S, a2*a3*(1-C) + a1*S, aa3 + (1-aa3)*C    ],              
  ])

  return R

### オイラー角（XYZ型）

各軸回転の行列を作っておく．

In [ ]:
def Rx(angle):
  return Rotation(angle,[1,0,0])

def Ry(angle):
  return Rotation(angle,[0,1,0])

def Rz(angle):
  return Rotation(angle,[0,0,1])

Rx(0.5), Ry(0.5), Rz(0.5)

XYZ型オイラー角による回転行列を作る．

In [ ]:
def Rxyz(angles):
  th1, th2, th3 = angles
  return Rz(th3).dot( Ry(th2) ).dot( Rx(th1) )

Rxyz( [0.5, 0.5, 0.5] )

### ポリゴンデータの操作

In [ ]:
def Rotate_polys(R, polys):
  '''
  回転行列 R を，ポリゴンのリスト polys に掛ける
  '''
  POLYs = []
  for poly in polys: #ポリゴン１枚の頂点セット in そのリスト
        
    POLY = []
    for point in poly: #各頂点 in あるポリゴンの頂点セット
      newpoint = R.dot(point)   #回転変換
      POLY.append(newpoint)     #POLYに格納
            
    POLYs.append(POLY) #POLYSに格納
        
  return POLYs

def plot_polys(ax, polys):
  '''
  ポリゴンのリスト polys をプロットする
  '''
  facecolors=['C1','C2','C6','C6','C3'] #角面の色

  for i, p in enumerate(polys):
    pc = Poly3DCollection([p])
    pc.set_alpha(0.7) #alpha first
    pc.set_facecolor(facecolors[i])
    ax.add_collection3d(pc)

  ax.set_xlim(-1,1)
  ax.set_ylim(-1,1)
  ax.set_zlim(-1,1)
  # ax.set_xlabel('X', labelpad=5)
  # ax.set_ylabel('Y', labelpad=5)
  # ax.set_zlabel('Z', labelpad=0)
  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')

### ポリゴンのデータサンプル

In [ ]:
def create_triangle_polys(w, d, h):
  '''
  厚みのある二等辺三角形 (幅, 厚, 高)
  '''
  # 頂点
  A = np.array([ w/2, -d/2, 0])
  B = np.array([ w/2,  d/2, 0])
  C = np.array([-w/2,  d/2, 0])
  D = np.array([-w/2, -d/2, 0])
  E = np.array([   0, -d/2, h])
  F = np.array([   0,  d/2, h])
    
  # ポリゴンのリスト
  polys =[
    [F, B, C],    #前面ポリゴン
    [E, D, A],    #後面ポリゴン
    [A, B, F, E], #斜面ポリゴン
    [C, D, E, F], #斜面ポリゴン
    [A, B, C, D], #底面ポリゴン
  ]
    
  return polys

### お試しプロット

In [ ]:
# 基準姿勢データのサンプル
body = create_triangle_polys(w=1, d=0.1, h=0.5)

# オイラー角による回転行列 R のサンプル
R = Rxyz( [pi/2, 0, 0] ) #X軸まわりにだけ回転

# 基準姿勢データを回転変換
newbody = Rotate_polys(R, body)

print(body)
print(newbody)

fig = plt.figure(figsize=(5,5))             #正方形のキャンバスに，
ax = fig.add_subplot(111, projection='3d')  #3次元グラフ用紙を作る

plot_polys(ax, body)
plot_polys(ax, newbody)

fig.tight_layout()
plt.show()

## 手動操作

【注意】簡易的な実装のため，滑らかには動きません．

In [ ]:
def draw_body(th1deg, th2deg, th3deg):
  '''
  オイラー角の姿勢で物体を描く関数
  '''
  fig = plt.figure(figsize=(5,5))             #正方形のキャンバスに，
  ax = fig.add_subplot(111, projection='3d')  #3次元グラフ用紙を作る

  ax.set_title('angles = (%d, %d, %d)'%(th1deg, th2deg, th3deg))
  angles_rad = np.array([th1deg, th2deg, th3deg])/180*pi

  # オイラー角による回転行列 R
  R = Rxyz( angles_rad )

  # 三角形のポリゴンデータ
  body = create_triangle_polys(w=1, d=0.1, h=0.5)

  # 三角形をオイラー角で回す
  newbody = Rotate_polys( R, body )

  # 描画する
  plot_polys(ax, newbody)
  fig.tight_layout()
  plt.show()

### UI(ユーザーインターフェイス)を作る
#スライダの定義
slider_th1 = ipw.IntSlider(description='th1', 
                min=-180, max=180, step=10, value=0) #下限,上限,刻み,初期値
slider_th2 = ipw.IntSlider(description='th2',
                min=-180, max=180, step=10, value=0)
slider_th3 = ipw.IntSlider(description='th3',
                min=-180, max=180, step=10, value=0)

#スライダを縦に並べるUI
ui = ipw.VBox([slider_th1, slider_th2, slider_th3])

#スライダの値でグラフを更新するオブジェクト
out = ipw.interactive_output( 
    draw_body, 
    { 
      'th1deg': slider_th1, 
      'th2deg': slider_th2, 
      'th3deg': slider_th3,
    }
)

out.layout.height = '350px' #サイズ固定で若干チラツキが緩和
out.layout.width  = '350px'
display(ui, out)            #UIとグラフを出力

- `th2deg = 90` に合わせると，ジンバルロックが発生します．
- このとき，`th1deg` と `th3deg` は，効き方が同じになります．